In [39]:
local_cur.close()
local_con.close()

In [38]:
remote_cur.close()
remote_con.close()

In [1]:
import cx_Oracle

In [2]:
import os, argparse, sys
import cx_Oracle
import getpass
import time
import sys

clarity_user = ''
clarity_psw = ''

dsnStr = cx_Oracle.makedsn("exaorat2-scan.chop.edu", "1521", service_name="CLTprd_pri.chop.edu")
remote_con = cx_Oracle.connect(user = clarity_user, password = clarity_psw, dsn = dsnStr)
remote_cur = remote_con.cursor()

In [3]:
# Find icd codes to check the patients have no psychiatric ICD codes

### Collect patient IDs by icd codes

remote_cmd = """
           
select p.pat_id, p.sex_c , p.birth_date from patient p

where p.BIRTH_DATE > to_date('31/DEC/2011','DD/MON/YYYY') and p.BIRTH_DATE < to_date('01/JAN/2013','DD/MON/YYYY')
                 
"""

remote_cur.execute(remote_cmd)

data = remote_cur.fetchall()

In [4]:
psy_dict = dict()

for d in data:
    pat_id = d[0]
    sex = d[1]
    birth = d[2]
    psy_dict[pat_id] = [sex,birth]
    
len(psy_dict)

75491

In [5]:
# Find icd codes to check the patients have no psychiatric ICD codes

### Collect patient IDs by icd codes

remote_cmd = """

with p as(  

                select dx.pat_id, dx.dx_id, icd.code
                from pat_enc_dx dx
                inner join pat_enc e on e.pat_enc_Csn_id = dx.pat_enc_Csn_id
                inner join edg_current_icd10 icd on icd.dx_Id = dx.dx_id
                
                union
                
                select dx.pat_id, dx.dx_id, icd.code
                from pat_enc_dx dx
                inner join pat_enc e on e.pat_enc_Csn_id = dx.pat_enc_Csn_id
                inner join edg_current_icd9 icd on icd.dx_Id = dx.dx_id
                

                union
                select h.pat_id, hdx.dx_id, edg.ref_bill_code
                from hsp_acct_dx_List hdx
                inner join pat_enc_hsp h on h.hsp_account_id = hdx.hsp_account_id
                inner join clarity_edg edg on edg.dx_Id = hdx.dx_Id
                where  edg.ref_bill_code_set_c in (1,2)

                

                union
                select pl.pat_id,pl.dx_id, icd.code
                from problem_List pl
                inner join edg_current_icd10 icd on icd.dx_Id = pl.dx_id
                where (pl.problem_status_c is null or pl.problem_status_c <> 3)
                
                union
                select pl.pat_id,pl.dx_id, icd.code
                from problem_List pl
                inner join edg_current_icd9 icd on icd.dx_Id = pl.dx_id
                where (pl.problem_status_c is null or pl.problem_status_c <> 3)

            )
            
select p.pat_id, p.code from p
inner join patient on patient.pat_id = p.pat_id

where patient.BIRTH_DATE > to_date('31/DEC/2011','DD/MON/YYYY') and patient.BIRTH_DATE < to_date('01/JAN/2013','DD/MON/YYYY')
                 

"""

remote_cur.execute(remote_cmd)

data = remote_cur.fetchall()

In [7]:
len(data)

1997900

In [8]:
pat_icd_codes_dict = dict()

for d in data:
    pat_id = d[0]
    #if(pat_id not in patient_05_to_15): continue
    icd = d[1]
    if(icd == None): continue
    if(pat_icd_codes_dict.get(pat_id) == None): pat_icd_codes_dict[pat_id] = set()
        
    pat_icd_codes_dict[pat_id].add(icd)    
    
len(pat_icd_codes_dict)

54518

In [9]:
def check_with_no_psy(pat_icd):
    
        
    psy_icd_check = True
    for icd in pat_icd:
        if(icd.startswith('F')):
            psy_icd_check = False
            break
            
        for i in range(290, 320):
            if(icd.startswith(str(i))):
                psy_icd_check = False
                break
                
        if(psy_icd_check == False): break
            
    return psy_icd_check

In [10]:
no_psy_pat_icd_codes_dict = dict()

for pat in pat_icd_codes_dict.keys():
    
    psy_icd_check = check_with_no_psy(pat_icd_codes_dict[pat])
    
    if(psy_icd_check ): no_psy_pat_icd_codes_dict[pat] = pat_icd_codes_dict[pat]
        
len(no_psy_pat_icd_codes_dict)

43633

In [21]:
nonasd = os.listdir('/mnt/isilon/wang_lab/mengge/project/autism_notes/noasdprob')

In [23]:

pat_need_deletion = []
for pat in no_psy_pat_icd_codes_dict.keys():
    if(pat in nonasd): pat_need_deletion.append(pat)

In [24]:
for pat in pat_need_deletion:
    no_psy_pat_icd_codes_dict.pop(pat)
    
len(no_psy_pat_icd_codes_dict)

45556

In [31]:
pat_icd9 = set()

for pat in no_psy_pat_icd_codes_dict.keys():
    
    if('F' in no_psy_pat_icd_codes_dict[pat]): print (pat)
        
    for i in range(290, 320):
        for icd in no_psy_pat_icd_codes_dict[pat]:
            if(icd.startswith(str(i))): pat_icd9.add(pat)
            break
len(pat_icd9)

0

In [12]:
### Collect patient IDs by icd codes

remote_cmd = """

with psy as   (

                select dx.pat_id, dx.dx_id
                from pat_enc_dx dx
                inner join pat_enc e on e.pat_enc_Csn_id = dx.pat_enc_Csn_id
                inner join edg_current_icd10 icd on icd.dx_Id = dx.dx_id
                where icd.code  like 'F%'

                union
                select h.pat_id, hdx.dx_id
                from hsp_acct_dx_List hdx
                inner join pat_enc_hsp h on h.hsp_account_id = hdx.hsp_account_id
                inner join clarity_edg edg on edg.dx_Id = hdx.dx_Id
                where ( edg.ref_bill_code  like 'F%')
                and edg.ref_bill_code_set_c in (1,2)

                

                union
                select pl.pat_id,pl.dx_id
                from problem_List pl
                inner join edg_current_icd10 icd on icd.dx_Id = pl.dx_id
                where icd.code  like 'F%'
                 and  (pl.problem_status_c is null or pl.problem_status_c <> 3)

                 ) -- inclusion diagnoses
                 
select distinct(p.pat_id), p.sex_c, p.birth_date
        from patient p
        inner join psy on psy.pat_id = p.pat_id
        inner join zc_sex  sex on sex.rcpt_mem_sex_c = p.sex_c
"""

remote_cur.execute(remote_cmd)

data = remote_cur.fetchall()

In [13]:
for p in psy_dict.keys():
    psy_dict[p].pop()

for d in data:
    pat_id = d[0]
    sex = d[1]
    birth = d[2]
    if(psy_dict.get(pat_id)):
        psy_dict[pat_id].append('psy')
    else:
        psy_dict[pat_id] = [sex,birth]
    
len(psy_dict)

1454320

In [14]:
psy_p_num = 0
for p in psy_dict:
    if(len(psy_dict[p]) > 2):psy_p_num += 1
psy_p_num

255160

In [20]:
nonasd = os.listdir('/mnt/isilon/wang_lab/mengge/project/autism_notes/noasdprob')

nonasd_nonpsy_exist = []

for pid in psy_dict.keys():
    if(len(psy_dict[pid]) < 3 and pid in nonasd):nonasd_nonpsy_exist.append(pid)
        
len(nonasd_nonpsy_exist)

KeyboardInterrupt: 

In [11]:
### Collect note IDs by patients

remote_cmd = """
            select pat_id, note_id from hno_info where 
"""
cnt = 0
#for p in nonasd_nonpsy_exist:
for p in no_psy_pat_icd_codes_dict.keys():
    if(cnt > 0):
        remote_cmd += ' or '
        
    cnt += 1
    
    remote_cmd += " pat_id = '{}' ".format(p)

#print(remote_cmd)    

remote_cur.execute(remote_cmd)

data = remote_cur.fetchall()

In [12]:
non_asd_non_psy_note_id = dict()

for d in data:
    pat_id = d[0]
    note_id = d[1]    
    if(non_asd_non_psy_note_id.get(pat_id) == None): non_asd_non_psy_note_id[pat_id] = set()
        
    non_asd_non_psy_note_id[pat_id].add(note_id)
    
len(non_asd_non_psy_note_id)

43420

In [13]:
remote_cmd_note = """
select hno_info.pat_id,  
       hno_info.CURRENT_AUTHOR_ID, 
       htext.note_id, htext.NOTE_TEXT, htext.LINE, htext.CONTACT_DATE,
        hno_info.IP_NOTE_TYPE_C
               
        from hno_info
        inner join hno_note_text htext on htext.note_id = hno_info.note_id
        where hno_info.pat_id='{}' and htext.CONTACT_DATE < to_date('01/JAN/2020','DD/MON/YYYY') and """

In [14]:

import sqlite3, os

local_db_name = 'nonpsy_nonasd_patient.db'

local_con = sqlite3.connect(local_db_name)

local_cur = local_con.cursor()

In [15]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
    finally:
        if c:
            c.close()


pat_id_table = """CREATE TABLE IF NOT EXISTS patients (
                                        pat_id text PRIMARY KEY,
                                        sex text,
                                        birth_date text
                                        
                                    ); """

note_tale = """CREATE TABLE IF NOT EXISTS note_text (
                                        pat_id text ,
                                        author_id text,
                                        auther_name text,
                                        contact_date text,
                                        note_text text,
                                        note_type text,
                                        note_id text);
                                        """

create_table(local_con, pat_id_table)

create_table(local_con, note_tale)

In [16]:
def local_sql_insert(conn, insert_cmd, insert_data):
    """
    Create a new task
    :param conn:
    :return:
    """
    cur = conn.cursor()
    cur.execute(insert_cmd, insert_data)
    #conn.commit()
    cur.close()
    return cur.lastrowid

In [17]:



pat_cnt = 0

note_err_log = 'note_err_log.1020_2008'

note_log = 'note_log.1020_2008'

processed_pat_id = []

try:
    processed_pat_id = [line.split(' ')[0] for line in open(note_log,'r')]
except:
    pass


### hno_info.CURRENT_AUTHOR_ID, emp.name, 
  ###     htext.note_id, htext.NOTE_TEXT, htext.LINE, htext.CONTACT_DATE
#for pat_id in nonasd_nonpsy_exist:
for pat_id in non_asd_non_psy_note_id.keys():   
    
    if(pat_id in processed_pat_id):
        continue
        #print('{} processed'.format(pat_id))
    """ searching_local_cmd = select count(*) from note_text where pat_id = '{}'.format(pat_id)
    
    local_cur.execute(searching_local_cmd)
    data = local_cur.fetchone()
    if(data[0] > 0):
        fa = open("note_log.0826", 'a+')
        fa.write('pat: {} queried for {} notes\n'.format(pat_id, str(data[0])))
        fa.close()
        continue"""

    """if(pat_id in queried_pat):
        fa = open("note_log.0901", 'a+')
        fa.write('pat: {} queried\n'.format(pat_id, str(data[0])))
        fa.close()
        continue"""
    if(psy_dict.get(pat_id) == None): continue
    sex = psy_dict[pat_id][0]
    birthdate = psy_dict[pat_id][1]
    
    try:
        
        
        local_insert_cmd_patients = ''' INSERT INTO patients (pat_id, sex, birth_date) VALUES(?,?, ?) '''
        local_insert_data = (pat_id, sex, birthdate)
        local_sql_insert(local_con, local_insert_cmd_patients,local_insert_data )
    except Exception as e:
        fa = open(note_err_log, 'a+')
        fa.write("Error in {}'\n".format(pat_id, pat_id))
        fa.write(str(e))
        fa.write('\n')
        fa.close()
        continue
    
    if(non_asd_non_psy_note_id.get(pat_id) == None): continue
        
    premote_cmd_note = remote_cmd_note.format(pat_id)
    premote_cmd_note += '('
    
    head = True
    
    for noteid in non_asd_non_psy_note_id[pat_id]:
        if(head):
            head = False
        else:
            premote_cmd_note += ' or '
        premote_cmd_note += " hno_info.note_id = '{}' ".format(noteid)
        
    premote_cmd_note += ')'
    """print(premote_cmd_note)
    break"""
    
    remote_cur.execute(premote_cmd_note)
    data = remote_cur.fetchall()
    note_dict = dict()
    
    """hno_info.pat_id,  
       hno_info.CURRENT_AUTHOR_ID, 
       htext.note_id, 
       htext.NOTE_TEXT,
       htext.LINE, 
       htext.CONTACT_DATE,
        hno_info.IP_NOTE_TYPE_C"""
    for d in data:
        try:
            author_id = str(d[1])
            
            note_id = str(d[2])
            note_text = d[3].replace('??', '\n')
            line_num = str(d[4])
            contact_date = d[5].strftime("%Y/%m/%d")
            note_type = str(d[6])
            if(note_dict.get(note_id) == None): note_dict[note_id] = dict()
            if(note_dict[note_id].get('author_id') == None): note_dict[note_id]['author_id'] = author_id
            #if(note_dict[note_id].get('author_name') == None): note_dict[note_id]['author_name'] = author_name
            if(note_dict[note_id].get('contact_date') == None): note_dict[note_id]['contact_date'] = contact_date
            if(note_dict[note_id].get('note_type') == None): note_dict[note_id]['note_type'] = note_type
            note_dict[note_id][int(line_num)] = note_text
        except Exception as e:
            fa = open(note_err_log, 'a+')
            fa.write("Error in {}, note id:{}\n".format(pat_id, note_id))
            fa.write(str(e))
            fa.write('\n')
            fa.close()
            
    for note_id in note_dict.keys():
        insert_text = ''
        for i in range(50):
            if(note_dict[note_id].get(i)): insert_text += note_dict[note_id][i]
        try:     
            local_insert_cmd_note_text  = ''' INSERT INTO note_text (pat_id ,author_id ,auther_name ,contact_date ,note_text ,note_type, note_id ) VALUES(?, ?,?,?,?,?, ?) '''   
            local_insert_data_note_text  = (pat_id, 
                                        note_dict[note_id]['author_id'] , 
                                        None,
                                        note_dict[note_id]['contact_date'],
                                        insert_text,
                                        note_dict[note_id]['note_type'],
                                        note_id
                                       )
        
            local_sql_insert(local_con, local_insert_cmd_note_text, local_insert_data_note_text )
        except Exception as e:
            fa = open(note_err_log, 'a+')
            fa.write("Error in {}, note id:{}\n".format(pat_id, note_id))
            fa.write(str(e))
            fa.write('\n')
            fa.close()
        
    pat_cnt += 1
    fa = open(note_log, 'a+')
    fa.write('{} processed\n'.format(pat_id))
    fa.close()
    if(pat_cnt % 100 ==0): 
        local_con.commit()
        print(pat_cnt)
    

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [1]:
local_con.commit()

NameError: name 'local_con' is not defined

In [2]:
header_set = set(['AST PSYCHIATRIC HISTORY',
 'CHILD PSYCHIATRISTS (ABLE TO PRESCRIBE MEDICATIONS)',
 'CONSULT TO PSYCHIATRY (CHOP)',
 'INITIAL INTAKE PSYCHIATRIC EVALUATION',
 'MD PSYCHIATRIST',
 'Next PSYCHIATRY NOTE',
 'PAST PSYCHIATRIC HISTORY',
 'PAST PSYCHIATRIC HISTORY ',
 'PAST PSYCHIATRIC HISTORY AT CHOP',
 'PAST PSYCHIATRIC MEDICATION',
 'PAST PSYCHIATRIC THERAPY',
 'PSYCH HISTORY',
 'PSYCHIATRIC CONSULTATION',
 'PSYCHIATRIC DIAGNOSTIC EVALUATION',
 'PSYCHIATRIC EVALUATION',
 'PSYCHIATRIC FOLLOW UP',
 'PSYCHIATRIC FOLLOW UP EVALUATION',
 'PSYCHIATRIC FOLLOW UP NOTE',
 'PSYCHIATRIC HISTORY',
 'PSYCHIATRIC RE-EVALUATION',
 'PSYCHIATRIC REVIEW OF SYMPTOMS',
 'PSYCHIATRIC REVIEW OF SYSTEMS',
 'PSYCHIATRIC REVIEW OF SYTEMS',
 'PSYCHIATRICnl',
 'PSYCHIATRIST',
 'PSYCHIATRIST FOR DIAGNOSIS',
 'PSYCHIATRISTS',
 'PSYCHIATRISTS AND PSYCHOLOGISTS',
 'PSYCHIATRY  FOLLOW UP  NOTE',
 'PSYCHIATRY  FOLLOW UP NOTE',
 'PSYCHIATRY CONSULT',
 'PSYCHIATRY CONSULT-LIAISON PROGRESS NOTE',
 'PSYCHIATRY CONSULTATION NOTE',
 'PSYCHIATRY FOLLOW UP  NOTE',
 'PSYCHIATRY FOLLOW UP NOTE',
 'PSYCHIATRY FOLLOW UPNOTE',
 'PSYCHIATRY FOLLOW-UP  NOTE',
 'PSYCHIATRY FOLLOW-UP PROGRESS NOTE',
 'PSYCHIATRY Follow -up note',
 'PSYCHIATRY Follow up NOTE',
 'PSYCHIATRY Follow up note',
 'PSYCHIATRY NOTE',
 'PSYCHIATRY PROGRESS FOLLOW-UP NOTE',
 'PSYCHIATRY PROGRESS NOTE',
 'Updated PSYCHIATRIC EVALUATION',
 'cmPSYCHIATRIC',
 'PSYCHIATRIC ATTENDING'
'PSYCHIATRIC REVIEW OF SYMPTOMS'
'PSYCHIATRIST FOR DIAGNOSIS'
'OUTPATIENT PSYCHIATRY'
'ATTENDING PSYCHIATRY ADDENDUM',
 'CHILD AND ADOLESCENT PSYCHIATRY ATTENDING',
 'CONTINUE ABILIFY 5 MG BID- FOLLOW UP PSYCHIATRY',
 'CONTINUE CURRENT OUTPATIENT PSYCHIATRIC MEDICATIONS',
 'CONTINUE OUTPATIENT PSYCHIATRY CARE',
 'CURRENT OUTPATIENT PSYCHIATRIC MEDICATION',
 'FOLLOW PSYCHIATRY APPT PENDING FOR MEDICATION EVALUATION',
 'FOLLOW UP AFTER DISCHARGE FROM PSYCHIATRY FACILITY',
 'FOLLOW UP DECEMBER 2018 OR PSYCHIATRY',
 'FOLLOW UP PSYCHOLOGICAL EVALUATION IN MAY',
 'HEALTHY MINDS, HEALTHY KIDS TELEPSYCHIATRY CONSULT NOTE',
 'INPATIENT PSYCHOLOGY PROGRESS NOTE',
 'MBU PSYCHIATRIC EVALUATION PROGRESS NOTE',
 'MBU PSYCHIATRY EVALUATION NOTE',
 'MBU PSYCHIATRY PROGRESS NOTE',
 'MBUPSYCHIATRY PROGRESS NOTE',
 'NEXT PSYCHIATRY NOTE',
 'NURSE PRACTITIONER, OUTPATIENT PSYCHIATRY',
 'ONGOING PSYCHIATRY FOLLOW UP.',
 'OUTPATIENT  PSYCHIATRIC CONSULTATION',
 'OUTPATIENT FOLLOW-UP PSYCHIATRY NOTE',
 'OUTPATIENT FOLLOW-UP PSYCHIATRY PROGRESS NOTE',
 'OUTPATIENT PSYCHIATRIC  EVALUATION',
 'OUTPATIENT PSYCHIATRIC  FOLLOW UP',
 'OUTPATIENT PSYCHIATRIC -FOLLOW UP',
 'OUTPATIENT PSYCHIATRIC CONSULTATION',
 'OUTPATIENT PSYCHIATRIC CONSULTATION EVALUATION',
 'OUTPATIENT PSYCHIATRIC EVALUATION',
 'OUTPATIENT PSYCHIATRIC EVALUATION AND CONSULTATION',
 'OUTPATIENT PSYCHIATRIC EVALUATION UPDATE',
 'OUTPATIENT PSYCHIATRIC FOLLOW -UP',
 'OUTPATIENT PSYCHIATRIC FOLLOW UP',
 'OUTPATIENT PSYCHIATRIC FOLLOW-UP APPOINTMENT',
 'OUTPATIENT PSYCHIATRIC FOLLOW-UP NOTE',
 'OUTPATIENT PSYCHIATRIC NOTE',
 'OUTPATIENT PSYCHIATRIC PROGRESS NOTE',
 'OUTPATIENT PSYCHIATRY CONSULTATION PROGRESS NOTE',
 'OUTPATIENT PSYCHIATRY EVALUATION',
 'OUTPATIENT PSYCHIATRY EVALUATION NOTE',
 'OUTPATIENT PSYCHIATRY FOLLOW-UP NOTE',
 'OUTPATIENT PSYCHIATRY FOLLOW-UP PROGRESS NOTE',
 'OUTPATIENT PSYCHIATRY NOTE',
 'OUTPATIENT PSYCHIATRY PROGRESS NOTE',
 'OUTPATIENT PSYCHIATRYFOLLOW-UP NOTE',
 'OUTPATIENT PSYCHOLOGY',
 'OUTPATIENT PSYCHOLOGY THERAPY NOTE',
 'OUTPATIENT THERAPY AND PSYCHIATRY THROUGH COMMUNITY COUNCIL',
 'PENN OUTPATIENT PSYCHIATRY',
 'PSYCHIATRIC  FOLLOW UP',
 'PSYCHIATRIC ATTENDING NOTE',
 'PSYCHIATRIC EVALUATION FOLLOW UP',
 'PSYCHIATRIC FOLLOW -UP EVALUATION',
 'PSYCHIATRIC FOLLOW UP',
 'PSYCHIATRIC FOLLOW UP  EVALUATION',
 'PSYCHIATRIC FOLLOW UP EVALUATION',
 'PSYCHIATRIC FOLLOW UP FOR MEDICATION AND MOOD MANAGEMENT',
 'PSYCHIATRIC FOLLOW UP NOTE',
 'PSYCHIATRIC HISTORY FOLLOWED OUTPATIENT',
 'PSYCHIATRIC MINIMALLY INTERACTIVE, FOLLOW MOTHERS COMMANDS',
 'PSYCHIATRIC NOTE AND',
 'PSYCHIATRIC SYMPTOMS',
 'PSYCHIATRY  FOLLOW UP  NOTE',
 'PSYCHIATRY  FOLLOW UP NOTE',
 'PSYCHIATRY AND PSYCHOLOGY FOLLOW UP',
 'PSYCHIATRY ATTENDING',
 'PSYCHIATRY ATTENDING CONSULT NOTE',
 'PSYCHIATRY ATTENDING NOTE',
 'PSYCHIATRY ATTENDING PROGRESS NOTE',
 'PSYCHIATRY BRIEF CONSULTATION NOTE',
 'PSYCHIATRY CONSULT NOTE',
 'PSYCHIATRY CONSULT-LIAISON PROGRESS NOTE',
 'PSYCHIATRY CONSULTATION NOTE',
 'PSYCHIATRY CONSULTATION NOTE - SHORT FORM',
 'PSYCHIATRY EVALUATION  NOTE',
 'PSYCHIATRY EVALUATION AND FOLLOW UP',
 'PSYCHIATRY FOLLOW -UP NOTE',
 'PSYCHIATRY FOLLOW UP',
 'PSYCHIATRY FOLLOW UP  NOTE',
 'PSYCHIATRY FOLLOW UP FOR MEDICATION MANAGEMENT',
 'PSYCHIATRY FOLLOW UP NOTE',
 'PSYCHIATRY FOLLOW UPNOTE',
 'PSYCHIATRY FOLLOW-UP  NOTE',
 'PSYCHIATRY FOLLOW-UP OUTPATIENT NOTE',
 'PSYCHIATRY FOLLOW-UP PROGRESS NOTE',
 'PSYCHIATRY NOTE',
 'PSYCHIATRY NOTE - EVALUATION FEEDBACK',
 'PSYCHIATRY NOTES',
 'PSYCHIATRY OUTPATIENT FOLLOW UP NOTE',
 'PSYCHIATRY OUTPATIENT FOLLOW-UP  NOTE',
 'PSYCHIATRY OUTPATIENT FOLLOW-UP NOTE',
 'PSYCHIATRY OUTPATIENT PROGRESS NOTE',
 'PSYCHIATRY PROGRESS FOLLOW-UP NOTE',
 'PSYCHIATRY PROGRESS NOTE',
 'PSYCHOLOGIC SYMPTOMS',
 'PSYCHOLOGICAL SYMPTOMS',
 'PSYCHOLOGY APPOINTMENT AND FOLLOW UP',
 'PSYCHOLOGY FOLLOW UP',
 'PSYCHOLOGY FOLLOW UP EVALUATION',
 'PSYCHOLOGY PROGRESS NOTE',
 'PSYCHOLOGY THERAPY NOTE'])

In [3]:
sql_cmd = """select distinct(pat_id) from note_text where ({}) and  length(note_text) > 2000 """
             #and (note_text not like '%ASD%' and note_text not like '%autis%')"""

pset = set()

note_dict = dict()
cnt = 0
for psy in header_set:
    if("'" in psy): continue
        
    sql_cmd1 = sql_cmd.format("note_text like'%{}%'".format(psy))


    local_cur.execute(sql_cmd1)
    data = local_cur.fetchall()
    print(str(cnt) + ' ' + psy)
    cnt += 1
    print(len(data))
    note_dict[psy] = set()
    for d in data:
        pset.add(d[0])
        note_dict[psy].add(d[0])

0 OUTPATIENT PSYCHIATRY PROGRESS NOTE
0
1 OUTPATIENT PSYCHIATRY FOLLOW-UP NOTE
0
2 OUTPATIENT PSYCHIATRIC  FOLLOW UP
0
3 OUTPATIENT PSYCHOLOGY
121
4 PSYCHIATRY NOTES
0
5 PAST PSYCHIATRIC HISTORY AT CHOP
0
6 PSYCHIATRY OUTPATIENT PROGRESS NOTE
0
7 PSYCHIATRIC FOLLOW UP NOTE
0
8 PSYCHIATRIC FOLLOW UP  EVALUATION
0
9 OUTPATIENT PSYCHIATRIC EVALUATION UPDATE
0
10 OUTPATIENT PSYCHIATRIC FOLLOW-UP NOTE
0
11 MBUPSYCHIATRY PROGRESS NOTE
0
12 OUTPATIENT PSYCHIATRYFOLLOW-UP NOTE
0
13 PSYCHIATRIC FOLLOW UP EVALUATION
0
14 NURSE PRACTITIONER, OUTPATIENT PSYCHIATRY
0
15 OUTPATIENT PSYCHIATRIC FOLLOW UP
0
16 PSYCHIATRY CONSULT
181
17 OUTPATIENT  PSYCHIATRIC CONSULTATION
0
18 MBU PSYCHIATRIC EVALUATION PROGRESS NOTE
0
19 HEALTHY MINDS, HEALTHY KIDS TELEPSYCHIATRY CONSULT NOTE
0
20 PSYCHIATRY OUTPATIENT FOLLOW UP NOTE
0
21 PSYCHIATRIC RE-EVALUATION
0
22 PSYCHIATRY FOLLOW-UP PROGRESS NOTE
0
23 PSYCHIATRISTS
234
24 PSYCHIATRIC EVALUATION FOLLOW UP
0
25 OUTPATIENT PSYCHIATRIC FOLLOW -UP
0
26 PSYCHIATRY F

In [4]:
len(pset)

3368

In [5]:
import os

path = '/mnt/isilon/wang_lab/mengge/project/autism_notes/nonpsy_nonasd_notes'

os.system('mkdir -p {}'.format(path))

0

In [6]:
sql_cmd = """select pat_id, author_id, note_id, note_text from note_text where note_text like '%{}%' and  length(note_text) > 1000 
             """

In [7]:
note_dict_sorted = sorted(note_dict.items(), key  =lambda kv: len(kv[1]), reverse=True)

In [ ]:
pat_note_cnt = dict()


for i in range(len(note_dict_sorted)):
    
    keyword = note_dict_sorted[i][0]
    print(keyword)
    local_cur.execute(sql_cmd.format(keyword))
    
    data = local_cur.fetchone()
    
    while(data):
        pat_id = data[0]
        
        os.system('mkdir -p {}/{}'.format(path, pat_id))
        
        author_id = data[1]
        note_id = data[2]
        #print(pat_id)
        if(pat_note_cnt.get(pat_id) == None): pat_note_cnt[pat_id] = 0
        #print(author_name)
        if(note_id == None): note_id = 'note_{}'.format(str(pat_note_cnt[pat_id]))
        #print(note_id)
        pat_note_cnt[pat_id] += 1
        
        
        note_text = data[3].replace('??', '\n')
        
        if(os.path.exists('{}/{}/{}'.format(path, pat_id, note_id))):
            with open('{}/{}/{}'.format(path, pat_id, note_id), 'a+') as fa:
                fa.write('### {}'.format(keyword))
                
        else:
            with open('{}/{}/{}'.format(path, pat_id, note_id), 'w+') as fw:
                fw.write(note_text)
                fw.write('\n\n### {}\n### {}'.format(str(author_id), keyword))
        
        data = local_cur.fetchone()
        

PSYCHIATRIST
PSYCHIATRIC HISTORY
PAST PSYCHIATRIC MEDICATION
PSYCHOLOGY PROGRESS NOTE
PSYCHIATRIC SYMPTOMS
PSYCHIATRIC EVALUATION
PSYCHIATRISTS
PSYCHIATRY CONSULT
PSYCHOLOGICAL SYMPTOMS
OUTPATIENT PSYCHOLOGY
AST PSYCHIATRIC HISTORY
PAST PSYCHIATRIC HISTORY
PSYCHOLOGIC SYMPTOMS
PSYCHIATRY PROGRESS NOTE
PSYCH HISTORY
PSYCHIATRY ATTENDING
PSYCHOLOGY FOLLOW UP
PSYCHOLOGY THERAPY NOTE
CHILD AND ADOLESCENT PSYCHIATRY ATTENDING
INPATIENT PSYCHOLOGY PROGRESS NOTE
PAST PSYCHIATRIC HISTORY 
PENN OUTPATIENT PSYCHIATRY
PSYCHIATRIC CONSULTATION
PSYCHIATRISTS AND PSYCHOLOGISTS
PSYCHIATRY FOLLOW UP
PSYCHIATRIC FOLLOW UP
CONSULT TO PSYCHIATRY (CHOP)
PSYCHIATRIC REVIEW OF SYMPTOMS
